In [1]:
import pandas as pd

Load the data

In [2]:
df_etaty = pd.read_csv('SIO etaty.csv')
df_staniny = pd.read_csv('staniny2022.csv')
df_subwencja = pd.read_csv('subwencja_2022.csv')
df_sio = pd.read_csv('SIO_30.09.2021.csv')

Normalize `staniny` column name.

In [3]:
df_staniny = df_staniny.rename(columns={'Staniny egzaminów 2022 r.': 'Nazwa szkoły / placówki'})

Clean, filter and rename `etaty` data.

In [4]:
cols_to_keep = [
    'Nazwa szkoły / placówki',
    'Etat - umowa',
    'Etat - godziny ponadwymiarowe',
    'Etat niesubwencyjny - formy zatrudnienia inne niż KN',
    'Etat niesubwencyjny - nieobecności',
    'Etat subwencyjny - ogółem',
    'Etat subwencyjny - bez stopnia awansu',
    'Etat subwencyjny - mianowany',
    'Etat subwencyjny - dyplomowany',
    'Etat niesubwencyjny - ogółem',
]

df_etaty = df_etaty[df_etaty['Typ szkoły / placówki'] == 'Szkoła podstawowa']
df_etaty = df_etaty[cols_to_keep]


Clean, filter and rename `SIO` data

In [5]:
# List of all P{} student categories
ps = [f"P{i}" for i in range(1,73)]

In [6]:
df_sio = df_sio.rename(columns={'Nazwa szkoły/placówki': 'Nazwa szkoły / placówki'})

cols_to_keep = [
    'Nazwa szkoły / placówki',
    'Liczba uczniów',
    'Liczba uczniów na wsi',
    'Liczba uczniów w mieście do 5tys.',
    'Liczba uczniów poza szkołą',
]
cols_to_keep.extend(ps)

df_sio = df_sio[df_sio['Typ szkoły/placówki'] == 'Szkoła podstawowa']
df_sio = df_sio[cols_to_keep]




---

Join data from `etaty`, `staniny` and `SIO` databases into one.

In [7]:
mini_merge_df = df_etaty.merge(df_staniny, on='Nazwa szkoły / placówki', how='inner')
mini_merge_df = mini_merge_df.merge(df_sio, on='Nazwa szkoły / placówki', how='inner')

Add `subwencja`

In [8]:
def get_float(row, field_name):
  vals = [float(str(val).replace(' ', '').replace(',', '.')) for val in row[field_name].values]
  if len(vals) > 1:
    return vals
  else:
    return vals[0]

Calculate `subwencja` / `il.uczniów` for each category

In [9]:
subw_per_type = {}

for p in ps:
  row = df_subwencja[df_subwencja['Kategoria uczniów (shortcut)'] == p]
  total_val = get_float(row, 'Kwota razem w zł')
  students = get_float(row, 'Uczniowie przelicz.')

  subw_per_stud = total_val/students if students else 0.0
  subw_per_type[p] = subw_per_stud

Merge `subwencja` data into all previous databases.

In [10]:
merge_df = mini_merge_df.copy()

In [11]:
for col, weight in subw_per_type.items():
    new_col = f"{col}_subwencja_total"
    merge_df[col] = get_float(merge_df, col)
    merge_df[new_col] = merge_df[col] * weight

/var/folders/4n/y8_w60j160g2dhf681gtcbk40000gn/T/ipykernel_30890/2281845581.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merge_df[new_col] = merge_df[col] * weight
/var/folders/4n/y8_w60j160g2dhf681gtcbk40000gn/T/ipykernel_30890/2281845581.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merge_df[new_col] = merge_df[col] * weight
/var/folders/4n/y8_w60j160g2dhf681gtcbk40000gn/T/ipykernel_30890/2281845581.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many

Normalize float values.

In [13]:
for col in merge_df.columns:
  if col != 'Nazwa szkoły / placówki':
    merge_df[col] = get_float(merge_df, col)

In [14]:
merge_df

,Nazwa szkoły / placówki,Etat - umowa,Etat - godziny ponadwymiarowe,Etat niesubwencyjny - formy zatrudnienia inne niż KN,Etat niesubwencyjny - nieobecności,Etat subwencyjny - ogółem,Etat subwencyjny - bez stopnia awansu,Etat subwencyjny - mianowany,Etat subwencyjny - dyplomowany,Etat niesubwencyjny - ogółem,...,P63_subwencja_total,P64_subwencja_total,P65_subwencja_total,P66_subwencja_total,P67_subwencja_total,P68_subwencja_total,P69_subwencja_total,P70_subwencja_total,P71_subwencja_total,P72_subwencja_total
0,SZKOŁA PODSTAWOWA NR 5 IM. HENRYKA SIENKIEWICZA,29.91,4.31,0.0,0.0,29.91,6.50,8.61,14.80,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SZKOŁA PODSTAWOWA NR 6 IM. MARII SKŁODOWSKIEJ ...,43.97,7.29,0.0,0.0,43.97,4.61,17.87,21.49,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SZKOŁA PODSTAWOWA NR 4 IM. MIKOŁAJA KOPERNIKA ...,25.43,4.94,0.0,0.0,25.43,1.95,8.78,14.70,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SZKOŁA PODSTAWOWA NR 9 IM.WŁ.JAGIEŁŁY W KUTNIE,66.71,16.92,0.0,2.0,64.71,8.56,8.55,47.60,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SZKOŁA PODSTAWOWA NR 1 IM.TADEUSZA KOŚCIUSZKI ...,92.66,14.91,0.0,5.0,87.66,10.00,25.33,52.33,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,SZKOŁA PODSTAWOWA NR 2 IM. MARSZAŁKA JÓZEFA PI...,30.51,3.43,0.0,0.0,30.51,5.00,3.83,21.68,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,SZKOŁA PODSTAWOWA NR 7 IM. ADAMA MICKIEWICZA,32.17,3.50,0.0,0.0,32.17,4.00,8.95,19.22,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

In [15]:
df_dochody = pd.read_csv('dochody_2022.csv')
df_wydatki = pd.read_csv('wydatki_2022.csv')

Clean and filter `dochody`, map school names to format used in the previous databases.

In [16]:
df_dochody = df_dochody.rename(columns={'Nazwa': 'Nazwa szkoły / placówki'})

names_to_replace = {
    "Szkoła Podstawowa Nr 5": "SZKOŁA PODSTAWOWA NR 5 IM. HENRYKA SIENKIEWICZA",
    "Szkoła Podstawowa Nr 6": "SZKOŁA PODSTAWOWA NR 6 IM. MARII SKŁODOWSKIEJ - CURIE W KUTNIE",
    "Szkoła Podstawowa Nr 4": "SZKOŁA PODSTAWOWA NR 4 IM. MIKOŁAJA KOPERNIKA W KUTNIE",
    "Szkoła Podstawowa Nr 9": "SZKOŁA PODSTAWOWA NR 9 IM.WŁ.JAGIEŁŁY W KUTNIE",
    "Szkoła Podstawowa Nr 1": "SZKOŁA PODSTAWOWA NR 1 IM.TADEUSZA KOŚCIUSZKI W KUTNIE",
    "Szkoła Podstawowa Nr 2": "SZKOŁA PODSTAWOWA NR 2 IM. MARSZAŁKA JÓZEFA PIŁSUDSKIEGO",
    "Szkoła Podstawowa Nr 7": "SZKOŁA PODSTAWOWA NR 7 IM. ADAMA MICKIEWICZA"
}
cols_to_keep = [
    'Nazwa szkoły / placówki',
    'DW',
    'Paragraf'
]
df_dochody.replace(names_to_replace, inplace=True)
df_dochody = df_dochody[cols_to_keep]

df_dochody['DW'] = get_float(df_dochody, 'DW')

Clean and filter `wydatki`, map school names to format used in the previous databases.

In [17]:
df_wydatki = df_wydatki.rename(columns={'Nazwa': 'Nazwa szkoły / placówki'})

cols_to_keep = [
    'Nazwa szkoły / placówki',
    'WW',
    'Paragraf'
]

df_wydatki.replace(names_to_replace, inplace=True)
df_wydatki = df_wydatki[cols_to_keep]

df_wydatki['WW'] = get_float(df_wydatki, 'WW')

---

Add school name shortcuts to all databases.

In [18]:
name_to_shortcut = {
    "SZKOŁA PODSTAWOWA NR 5 IM. HENRYKA SIENKIEWICZA": "Sp5",
    "SZKOŁA PODSTAWOWA NR 6 IM. MARII SKŁODOWSKIEJ - CURIE W KUTNIE": "Sp6",
    "SZKOŁA PODSTAWOWA NR 4 IM. MIKOŁAJA KOPERNIKA W KUTNIE": "Sp4",
    "SZKOŁA PODSTAWOWA NR 9 IM.WŁ.JAGIEŁŁY W KUTNIE": "Sp9",
    "SZKOŁA PODSTAWOWA NR 1 IM.TADEUSZA KOŚCIUSZKI W KUTNIE": "Sp1",
    "SZKOŁA PODSTAWOWA NR 2 IM. MARSZAŁKA JÓZEFA PIŁSUDSKIEGO": "Sp2",
    "SZKOŁA PODSTAWOWA NR 7 IM. ADAMA MICKIEWICZA": "Sp7",
}

In [19]:
def add_shortcut(df):
  df['Nazwa_short'] = df['Nazwa szkoły / placówki'].map(name_to_shortcut)

In [20]:
add_shortcut(df_dochody)
add_shortcut(df_wydatki)
add_shortcut(merge_df)

/var/folders/4n/y8_w60j160g2dhf681gtcbk40000gn/T/ipykernel_30890/2673255479.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Nazwa_short'] = df['Nazwa szkoły / placówki'].map(name_to_shortcut)


In [21]:
df_dochody.to_csv('dochody_clean.csv')
df_wydatki.to_csv('wydatki_clean.csv')
merge_df.to_csv('merge.csv')